In [2]:
import numpy as np

from scipy.integrate import solve_ivp

from bokeh.io import  output_notebook, push_notebook, show
from bokeh.plotting import figure
from bokeh.layouts import column, row
from bokeh.models import Label

from ipywidgets import interact, FloatSlider

from mylib.model import brusselator_model
import mylib.continuation as continuation

output_notebook(hide_banner=True)

# Brusselator model

The dynamics of the oscillating reaction discovered by Belousov and Zhabotinsky,
can be modeled through the so-called Brusselator model depending on two parameters:
$$
\left\{\begin{aligned}
{\mathrm d} y_1 & = a - (b+1) y_1 + y_1^2y_2\\
{\mathrm d} y_2 & = b y_1 - y_1^2y_2
\end{aligned}\right.
$$

For this PC on equlibrium study and continuation, we will set $b=3$ and work with $a$ as a bifurcation parameter in a neighbor of $\sqrt{2}$, for which a Hopf bifurcation is taking place.

In [3]:
def plot_sol():
    
    bm = brusselator_model(a=1, b=3.)
    fcn = bm.fcn
    jac_eq = bm.jac_eq
    
    tini = 0. 
    tend = 80.
    
    yini = (1.5, 0)
    
    tol = 1.e-10
    sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=tol, atol=tol)
    eig_vals_eq, _= np.linalg.eig(jac_eq())
    
    fig_sol = figure(x_range=(tini, tend), width=950, height=300, title="Solution")
    plt_y1 = fig_sol.line(sol.t, sol.y[0], legend_label="y1", line_width=2)    
    plt_y2 = fig_sol.line(sol.t, sol.y[1], legend_label="y2", line_width=2, color="green")
    fig_sol.legend.location = "top_left"
    
    fig_pha = figure(plot_height=300, plot_width=475, title="Phase plan")
    plt_pha = fig_pha.line(sol.y[0], sol.y[1], line_width=2)

    fig_eig = figure(x_range=(-3.5, 3.5), y_range=(-2, 2), plot_height=300, plot_width=475, 
                     title="Eigen values at equilibrium points")
    fig_eig.line(x=0, y=np.linspace(-2.,2,10), color="black")
    fig_eig.line(x=np.linspace(-3.5,3.5,10), y=0, color="black")
    plt_eig = fig_eig.x(np.real(eig_vals_eq), np.imag(eig_vals_eq), line_width=2, size=10)

    show(column(fig_sol, row(fig_pha, fig_eig)), notebook_handle=True)
    
    def update(a):
        bm = brusselator_model(a, b=3.)
        fcn = bm.fcn
        jac_eq = bm.jac_eq
        sol = solve_ivp(fcn, (tini, tend), yini, method="RK45", rtol=tol, atol=tol)
        eig_vals_eq, _= np.linalg.eig(jac_eq())
        plt_y1.data_source.data = dict(x=sol.t, y=sol.y[0])
        plt_y2.data_source.data = dict(x=sol.t, y=sol.y[1])
        plt_pha.data_source.data = dict(x=sol.y[0], y=sol.y[1])
        plt_eig.data_source.data = dict(x=np.real(eig_vals_eq), y=np.imag(eig_vals_eq))
        push_notebook()

    interact(update, a=FloatSlider(min=0.4, max=3, value=1, step=0.01, 
                                   continuous_update=False))

plot_sol()

interactive(children=(FloatSlider(value=1.0, continuous_update=False, description='a', max=3.0, min=0.4, step=…

## Natural parameter continuation

In [4]:
def show_natural_continuation():
    
    a_ini = 0.1
    a_end = 3
    na = 201
    
    yeq_ini = (a_ini, 3/a_ini)

    yeq = continuation.natural_continuation_order_0(a_ini, a_end, na, yeq_ini, brusselator_model)
    #yeq = continuation.natural_continuation_order_1(a_ini, a_end, na, yeq_ini, brusselator_model)
    
    a = np.linspace(a_ini, a_end, na)
    a_stable   = a[np.where(a > np.sqrt(2))]
    yeq_stable   = yeq[:, np.where(a > np.sqrt(2))[0]]
    a_unstable   = a[np.where(a < np.sqrt(2))]
    yeq_unstable = yeq[:, np.where(a < np.sqrt(2))[0]]
    yhopf = 0.5*(yeq_stable[:,0] + yeq_unstable[:,-1])
    ahopf = 0.5*(a_stable[0] + a_unstable[-1])
    
    fig_branch_y1 = figure(plot_height=450, plot_width=950)
    fig_branch_y1.xaxis.axis_label = "a"
    fig_branch_y1.yaxis.axis_label = "y1"    

    fig_branch_y1.x(a_stable, yeq_stable[0], color="green", legend_label="stable branch for y1")
    fig_branch_y1.x(a_unstable, yeq_unstable[0], color="red", legend_label="unstable branch for y1")
    fig_branch_y1.x(ahopf, yhopf[0], size=15, line_width=3, legend_label="Hopf bifurcation")
    fig_branch_y1.legend.location = "top_left"

    fig_branch_y2 = figure(plot_height=450, plot_width=950)
    fig_branch_y2.xaxis.axis_label = "a"
    fig_branch_y2.yaxis.axis_label = "y2"    

    fig_branch_y2.x(a_stable, yeq_stable[1], color="green", legend_label="stable branch for y2")
    fig_branch_y2.x(a_unstable, yeq_unstable[1], color="red", legend_label="unstable branch for y2")
    fig_branch_y2.x(ahopf, yhopf[1], size=15, line_width=3, legend_label="Hopf bifurcation")

    show(column(fig_branch_y1, fig_branch_y2))

show_natural_continuation()